In [1]:
#import pylas
import numpy as np
import laspy
import open3d as o3d


las = laspy.read(r'C:\Users\frede\Desktop\master-thesis\data\PUNKTSKY_00005_1km_6079_522.laz')
las

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


<LasData(1.4, point fmt: <PointFormat(7, 19 bytes of extra dims)>, 17708868 points, 3 vlrs)>

In [2]:
las.header
las.header.point_format
las.header.point_count
las.vlrs

[<ExtraBytesVlr(extra bytes structs: 4)>, <laspy.vlrs.known.WktCoordinateSystemVlr object at 0x000001FD3FDC67C0>, <VLR(user_id: 'liblas', record_id: '2112', data len: 905)>]

In [3]:
list(las.point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'synthetic',
 'key_point',
 'withheld',
 'overlap',
 'scanner_channel',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'user_data',
 'scan_angle',
 'point_source_id',
 'gps_time',
 'red',
 'green',
 'blue',
 'ClassFlags',
 'Amplitude',
 'Reflectance',
 'Deviation']

In [4]:
set(list(las.classification))

{2, 3, 4, 5, 6, 7, 8, 9, 18}

In [3]:
point_data = np.stack([las.X, las.Y, las.Z], axis=0).transpose((1, 0))
point_data.shape

(17708868, 3)

In [ ]:
geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(point_data)
o3d.visualization.draw_geometries([geom])






In [6]:
# Seperate the classes from set(list(las.classification))

buildings = laspy.create(point_format=las.header.point_format, file_version=las.header.version)
buildings.points = las.points[las.classification == 6]

two = laspy.create(point_format=las.header.point_format, file_version=las.header.version)
two.points = las.points[las.classification == 2]

buildings.write('buildings.las')
two.write('two.las')

In [7]:
geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(point_data)
o3d.visualization.draw_geometries([geom])
